# Const Check fail

Here is a sdfg that leads to compilation error using the legacy CUDACodeGen. Below is the sdfg.

In [1]:
import dace

sdfg = dace.SDFG.from_file("../yakups_examples/weird_global_to_global.sdfg")
sdfg

SDFG (kernel)

Ensure you are using the legay CUDACodegen

In [ ]:
from dace.config import Config

assert Config.get('compiler', 'cuda', 'implementation') == 'legacy', "Select legacy CUDACodgen in config.yml & restart jupyter notebook"

Now let's try to compile the sdfg and observe how it fails:

In [3]:
sdfg.compile()

CompilationError: Compiler failure:
[ 16%] [32mBuilding CXX object CMakeFiles/kernel.dir/home/berkay/master-thesis/dace/berkay_workpace/scratch/thesis_related/.dacecache/kernel/src/cpu/kernel.cpp.o[0m
In file included from /home/berkay/master-thesis/dace/dace/codegen/../runtime/include/dace/dace.h:14,
                 from /home/berkay/master-thesis/dace/berkay_workpace/scratch/thesis_related/.dacecache/kernel/src/cpu/kernel.cpp:2:
/home/berkay/master-thesis/dace/dace/codegen/../runtime/include/dace/types.h: In constructor ‘dace::half::half(float)’:
/home/berkay/master-thesis/dace/dace/codegen/../runtime/include/dace/types.h:101:28: warning: dereferencing type-punned pointer will break strict-aliasing rules [-Wstrict-aliasing]
  101 |             uint32_t x = *((uint32_t*)&f);
      |                           ~^~~~~~~~~~~~~~
[ 33%] [32mBuilding CUDA object CMakeFiles/kernel.dir/home/berkay/master-thesis/dace/berkay_workpace/scratch/thesis_related/.dacecache/kernel/src/cuda/kernel_c

### ❓ Why does this fail?

The error is a **compilation failure** when calling the function template `dace::GlobalToGlobal1D`, due to a mismatch in argument types.

The function expects:

```cpp
(const T* src, int src_xstride, T* dst)
```

But in the following example:

```cpp
dace::GlobalToGlobal1D<double, 256, 1, 1, 256, 1, false>(A + i, 1, A);
```

both the **source** (`A + i`) and **destination** (`A`) point to the **same array**, just at different locations.

---

### 🧠 Why is this a problem?

1. **Wrong `const` deduction**  
   The old code generator mistakenly marks `A` as `const`, even though it is used as the **destination**.

2. **Missing overloads**  
   The template should allow for cases where the destination is not `const`. Overloading should be used to handle this properly — even if it is probably not 
   strictly an error, it is probably good practice
